<a href="https://colab.research.google.com/github/GBieler/colabNotebooks/blob/main/NetworkFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import pandas as pd
df = pd.read_csv('/content/titanic.csv')
df['const'] = 1
df.head()

,Survived,SibSp,Parch,Age_N,log_Fare,Pclass_1,Pclass_2,Embark_S,Embark_C,Male,const
0,0,1,0,0.28,0.92,0,0,1,0,1,1
1,1,1,0,0.48,1.86,1,0,0,1,0,1
2,1,0,0,0.33,0.95,0,0,1,0,0,1
3,1,1,0,0.44,1.73,1,0,1,0,0,1
4,0,0,0,0.44,0.96,0,0,1,0,1,1


In [127]:
import torch.nn.functional as F

def rectified_linear(p,x):
  return F.relu(p*x)

def relu(p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,x):
    x1= x[0]
    x2= x[1]
    x3= x[2]
    x4= x[3]
    x5= x[4]
    x6= x[5]
    x7= x[6]
    x8= x[7]
    x9= x[8]
    x10= x[9]
    return rectified_linear(p1,x1) + rectified_linear(p2,x2) + rectified_linear(p3,x3) + rectified_linear(p4,x4) + rectified_linear(p5,x5) + rectified_linear(p6,x6) + rectified_linear(p7,x7) + rectified_linear(p8,x8) + rectified_linear(p9,x9) + rectified_linear(p10,x10)

def mk_rectified_linear(p1,p2,p3,p4,p5,p6,p7,p8,p9,p10):
  return partial(relu, p1,p2,p3,p4,p5,p6,p7,p8,p9,p10)

def mae(preds, acts):
  return (torch.abs(preds-acts)).mean()

def relu_mae(params):
    f1 = mk_rectified_linear(*params[:,0])
    f2 = mk_rectified_linear(*params[:,1])
    return mae(f1(x)+f2(x), y)

def get_data(rowIndex):
  row = df.iloc[rowIndex]
  x = list(row[1:])
  y = row[0].item()
  return x, y

In [128]:
total_loss = []

torch.manual_seed(seed=24)
params = torch.rand(10,2)
params.requires_grad_()

epoch = 2

for i in range(epoch):
  print(f'pass {i+1}')
  for rowIndex in range(len(df)):
      x, y =  get_data(rowIndex)
      loss = relu_mae(params)
      loss.backward()
      with torch.no_grad():
        params -= params.grad*0.00001
      total_loss += [loss.item()]
  print('loss:', sum(total_loss)/len(total_loss))

print('params', params)

pass 1
loss: 1.9775948926716196
pass 2
loss: 1.1902512968449985
params tensor([[-2.2507, -2.0734],
        [-0.9750, -1.7528],
        [-1.2490, -1.4470],
        [-3.5166, -5.0568],
        [-0.8557, -0.9834],
        [-0.8599, -0.7872],
        [-1.5503, -2.5458],
        [-0.0885, -0.7548],
        [-2.8407, -2.1595],
        [-2.8672, -3.0255]], requires_grad=True)
